# Generación de los textos facilitados

### Modelos
Usamos los siguientes modelos para generar los textos:
- [LLaMA3.1-8B Instruct](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct)
- [Mistral-8B Instruct](https://huggingface.co/mistralai/Ministral-8B-Instruct-2410)
- [Phi-3-Small-128K Instruct](https://huggingface.co/microsoft/Phi-3-small-128k-instruct)
- [Qwen2.5-7B Instruct](https://huggingface.co/Qwen/Qwen2.5-7B-Instruct)
- [DeepSeek-R1 Distill Qwen3-8B](https://huggingface.co/deepseek-ai/DeepSeek-R1-0528-Qwen3-8B)

Para realizar las peticiones mediante las APIs utilizo el archivo `config.json`, que contiene los tokens de todos los servicios que utilizo.

Por motivos de seguridad este archivo no está incluido en este repositorio.

In [ ]:
import os
import json
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from mistralai import Mistral
from huggingface_hub import InferenceClient
import re


PROMPT_LECTURA_FACIL = (
    "Eres un experto en accesibilidad y simplificación de textos. Tu tarea es transformar el siguiente texto para que sea comprensible por cualquier persona, incluyendo personas con discapacidad intelectual, dificultades lectoras o bajo nivel de alfabetización.\n"
    "Debes aplicar TODAS las pautas de Lectura Fácil de forma estricta y priorizar la claridad, sencillez y accesibilidad. No omitas ni ignores ninguna regla.\n"
    "No repitas el texto original ni añadas explicaciones. Responde únicamente con el texto simplificado.\n"
    "\nPAUTAS DE ORTOTIPOGRAFÍA:\n"
    "- No uses mayúsculas en palabras o frases completas, excepto en siglas.\n"
    "- Usa mayúscula inicial solo al inicio de párrafos, títulos, después de punto o en nombres propios.\n"
    "- Separa ideas diferentes con punto y aparte, no con coma.\n"
    "- Usa punto y aparte o conjunciones en vez de punto y seguido o coma para ideas relacionadas.\n"
    "- Usa dos puntos (:) para listas de más de tres elementos.\n"
    "- No uses punto y coma (;).\n"
    "- Evita paréntesis, corchetes y signos poco habituales (% & / ...).\n"
    "- No uses etcétera ni puntos suspensivos (...), reemplázalos por 'entre otros' o 'y muchos más'.\n"
    "- Evita comillas; si las usas, acompáñalas de explicación.\n"
    "\nPAUTAS DE VOCABULARIO:\n"
    "- Usa lenguaje sencillo y frecuente, adaptado al público objetivo.\n"
    "- Evita términos abstractos, técnicos o complejos.\n"
    "- Sustituye palabras homófonas/homógrafas por sinónimos.\n"
    "- Evita palabras largas o con sílabas complejas.\n"
    "- Evita adverbios terminados en -mente.\n"
    "- Evita superlativos, usa 'muy' + adjetivo.\n"
    "- Elimina palabras redundantes o innecesarias.\n"
    "- Evita palabras en otros idiomas salvo uso común (ej. wifi).\n"
    "- No uses abreviaturas ni siglas sin explicar la primera vez.\n"
    "- Evita frases nominales y lenguaje figurado (o explícalo).\n"
    "- Usa siempre la misma palabra para el mismo referente.\n"
    "- Evita palabras indeterminadas (cosa, algo).\n"
    "- Escribe los números con cifras; para números grandes, usa comparaciones cualitativas.\n"
    "- Separa los dígitos de teléfonos por bloques.\n"
    "- Evita números ordinales, usa cardinales.\n"
    "- Evita fracciones y porcentajes, usa descripciones equivalentes.\n"
    "- Escribe fechas completas (ej. 'el 1 de enero de 2023').\n"
    "- Usa el formato de 12 horas con texto (ej. 'las 3 de la tarde').\n"
    "- Evita números romanos, escríbelos como se leen.\n"
    "\nPAUTAS DE ORACIONES:\n"
    "- Usa frases sencillas, evita oraciones complejas.\n"
    "- Usa presente de indicativo siempre que sea posible.\n"
    "- Evita tiempos compuestos, condicionales y subjuntivos.\n"
    "- Usa voz activa, evita la pasiva y la pasiva refleja.\n"
    "- Usa imperativo solo en contextos claros, aclarando a quién se dirige.\n"
    "- Evita oraciones impersonales y con gerundio.\n"
    "- Evita verbos consecutivos salvo perífrasis con deber, querer, saber o poder.\n"
    "- Prefiere oraciones afirmativas, evita doble negación.\n"
    "- No uses elipsis, expresa todas las ideas claramente.\n"
    "- Evita explicaciones entre comas o aposiciones que corten el ritmo.\n"
    "- Limita las oraciones a dos ideas por frase como máximo.\n"
    "- Usa conectores simples, evita conectores complejos como 'por lo tanto' o 'sin embargo'.\n"
    "\n\nTexto: \n{texto}\nSimplificación: "
)


def get_api_key(model_type, config_file='config.json'):
    try:
        with open(config_file, 'r') as f:
            config = json.load(f)
    except FileNotFoundError:
        raise RuntimeError(f"Error: No se encontró el archivo {config_file}")
    except json.JSONDecodeError:
        raise RuntimeError(f"Error: El archivo {config_file} no tiene un formato JSON válido")
    
    key_name = f"{model_type.upper()}_API_KEY"
    api_key = config.get(key_name)
    
    if not api_key or api_key == f"your_{model_type.lower()}_api_key_here":
        raise RuntimeError(f"Error: La API key de {model_type} no esta en {config_file}")
    
    return api_key


def get_model_client(model_type):
    if model_type.lower() == 'mistral':
        api_key = get_api_key('mistral')
        return Mistral(api_key=api_key)
    elif model_type.lower() == 'huggingface':
        api_key = get_api_key('huggingface')
        return InferenceClient(token=api_key)
    else:
        raise ValueError(f"Tipo de modelo no soportado: {model_type}")


def simplificar_texto_mistral(texto_original):
    try:
        client = get_model_client('mistral')
        model = "mistral-large-latest"
        prompt = PROMPT_LECTURA_FACIL.format(texto=texto_original)
        max_tokens = 2 * len(texto_original.split())
        response = client.chat.complete(
            model=model,
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            max_tokens=max_tokens,
            temperature=0.3
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error en la respuesta del modelo: {str(e)}"


def simplificar_texto_llama3_8b(texto_original):
    try:
        client = get_model_client('huggingface')
        client.provider = "nebius"
        max_tokens = 2 * len(texto_original.split())
        response = client.chat.completions.create(
            model="meta-llama/Llama-3.1-8B-Instruct",
            messages=[
                {
                    "role": "system",
                    "content": "Eres un asistente experto que ayuda a simplificar textos aplicando correctamente las pautas de Lectura Fácil."
                },
                {
                    "role": "user",
                    "content": PROMPT_LECTURA_FACIL.format(texto=texto_original)
                }
            ],
            temperature=0.3,
            max_tokens=max_tokens
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error en la respuesta del modelo Llama3: {str(e)}"


def simplificar_texto_phi(texto_original):
    try:
        endpoint = "https://models.github.ai/inference"
        model_name = "microsoft/Phi-3-small-128k-instruct"
        token = get_api_key('github')
        max_tokens = 2 * len(texto_original.split())
        client = ChatCompletionsClient(
            endpoint=endpoint,
            credential=AzureKeyCredential(token),
        )
        response = client.complete(
            messages=[
                SystemMessage(
                    content="Eres un asistente experto que ayuda a simplificar textos aplicando correctamente las pautas de Lectura Fácil."
                ),
                UserMessage(
                    content=PROMPT_LECTURA_FACIL.format(texto=texto_original)
                ),
            ],
            temperature=0.3,
            top_p=1.0,
            max_tokens=max_tokens,
            model=model_name
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error en la respuesta del modelo Llama3: {str(e)}"


def simplificar_texto_qwen25(texto_original):
    try:
        client = get_model_client('huggingface')
        client.provider = "together"
        max_tokens = 2 * len(texto_original.split())
        response = client.chat.completions.create(
            model="Qwen/Qwen2.5-7B-Instruct",
            messages=[
                {
                    "role": "system",
                    "content": "Eres un asistente experto que ayuda a simplificar textos aplicando correctamente las pautas de Lectura Fácil."
                },
                {
                    "role": "user",
                    "content": PROMPT_LECTURA_FACIL.format(texto=texto_original)
                }
            ],
            temperature=0.3,
            max_tokens=max_tokens
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error en la respuesta del modelo Qwen: {str(e)}"


def simplificar_texto_deepseek(texto_original):
    try:
        client = get_model_client('huggingface')
        client.provider = "novita"
        max_tokens = 2 * len(texto_original.split())
        response = client.chat.completions.create(
            model="deepseek-ai/DeepSeek-R1-0528-Qwen3-8B",
            messages=[
                {
                    "role": "system",
                    "content": "Eres un asistente experto que ayuda a simplificar textos aplicando correctamente las pautas de Lectura Fácil."
                },
                {
                    "role": "user",
                    "content": PROMPT_LECTURA_FACIL.format(texto=texto_original)
                }
            ],
            temperature=0.3,
            max_tokens=max_tokens
        )
        texto = response.choices[0].message.content
        texto_filtrado = re.sub(r'<think>[\s\S]*?</think>', '', texto)
        return texto_filtrado.strip()
    except Exception as e:
        return f"Error en la respuesta del modelo DeepSeek: {str(e)}"


def simplificar_texto(texto_original, modelo='mistral'):
    if modelo.lower() == 'mistral':
        return simplificar_texto_mistral(texto_original)
    elif modelo.lower() == 'llama3':
        return simplificar_texto_llama3_8b(texto_original)
    elif modelo.lower() == 'phi':
        return simplificar_texto_phi(texto_original)
    elif modelo.lower() == 'qwen25':
        return simplificar_texto_qwen25(texto_original)
    elif modelo.lower() == 'deepseek':
        return simplificar_texto_deepseek(texto_original)
    else:
        raise ValueError(f"Modelo no soportado: {modelo}")


def procesar_textos_json():
    try:
        with open('500_facilitadas.json', 'r', encoding='utf-8') as f:
            textos = json.load(f)
    except Exception as e:
        print(f"Error leyendo el archivo de entrada: {str(e)}")
        return

    resultados = []
    try:
        for i, entrada in enumerate(textos):
            print(f"Procesando texto {i+1}/{len(textos)}...")
            resultado = {
                "original": entrada.get("TXT", ""),
                "facilitado": entrada.get("FAC", ""),
                "mistral": simplificar_texto(entrada.get("TXT", ""), "mistral"),
                "llama3": simplificar_texto(entrada.get("TXT", ""), "llama3"),
                "phi": simplificar_texto(entrada.get("TXT", ""), "phi"),
                "qwen25": simplificar_texto(entrada.get("TXT", ""), "qwen25"),
                "deepseek": simplificar_texto(entrada.get("TXT", ""), "deepseek")
            }
            resultados.append(resultado)
    except Exception as e:
        print(f"Error durante el procesamiento: {str(e)}")
    finally:
        try:
            with open('resultados.json', 'w', encoding='utf-8') as f:
                json.dump(resultados, f, ensure_ascii=False, indent=2)
            print(f"Resultados guardados (parciales o completos) en resultados.json")
        except Exception as e2:
            print(f"Error guardando los resultados: {str(e2)}")


if __name__ == "__main__":
    procesar_textos_json()

# Evaluación de los modelos

### Métricas
Para evaluar los modelos, utilizamos las siguientes métricas:
- **BLEU**: Mide la similitud entre el texto generado y el texto de referencia.
- **ROUGE**: Evalúa la calidad del resumen generado comparándolo con un resumen de referencia.
- **METEOR**: Mide la similitud semántica entre el texto generado y el texto de referencia.
- **BERTScore**: Utiliza embeddings de BERT para medir la similitud entre el texto generado y el texto de referencia.

## BLEU

In [ ]:
import json
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
import numpy as np
nltk.download('punkt_tab')

def cargar_resultados(archivo='resultados.json'):
    with open(archivo, 'r', encoding='utf-8') as f:
        return json.load(f)

def calcular_bleu(texto_generado, texto_referencia):
    referencia = word_tokenize(texto_referencia.lower(), language='spanish')
    candidato = word_tokenize(texto_generado.lower(), language='spanish')
    
    return sentence_bleu([referencia], candidato, weights=(0.25, 0.25, 0.25, 0.25))

def evaluar_modelos():
    try:
        nltk.data.find('tokenizers/punkt')
    except LookupError:
        nltk.download('punkt')
    
    resultados = cargar_resultados()
    
    scores_por_modelo = {
        'mistral': [],
        'llama3': [],
        'phi': [],
        'qwen25': [],
        'deepseek': []
    }
    
    for entrada in resultados:
        texto_facilitado = entrada['facilitado']
        
        for modelo in scores_por_modelo.keys():
            if modelo in entrada and entrada[modelo] and not entrada[modelo].startswith('Error') and not entrada[modelo].isspace() and not entrada[modelo] == '':
                try:
                    score = calcular_bleu(entrada[modelo], texto_facilitado)
                    scores_por_modelo[modelo].append(score)
                except LookupError as e:
                    print(f"Error de tokenización para el modelo {modelo}: {e}")
    
    print("\nResultados de evaluación BLEU:")
    print("-" * 50)
    print("Modelo\t\tMedia\t\tDesv. Est.\tMín\t\tMáx")
    print("-" * 50)
    
    for modelo, scores in scores_por_modelo.items():
        if scores:
            stats = {
                'media': np.mean(scores),
                'std': np.std(scores),
                'min': np.min(scores),
                'max': np.max(scores)
            }
            print(f"{modelo:<12}\t{stats['media']:.4f}\t\t{stats['std']:.4f}\t\t{stats['min']:.4f}\t\t{stats['max']:.4f}")
        else:
            print(f"{modelo:<12}\tNo hay resultados válidos")

if __name__ == "__main__":
    evaluar_modelos()

## ROUGE

In [ ]:
import json
import numpy as np
from rouge_score import rouge_scorer

def cargar_resultados(archivo='resultados.json'):
    with open(archivo, 'r', encoding='utf-8') as f:
        return json.load(f)

def calcular_rouge(texto_generado, texto_referencia):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(texto_referencia, texto_generado)
    return {
        'rouge1': scores['rouge1'].fmeasure,
        'rouge2': scores['rouge2'].fmeasure,
        'rougeL': scores['rougeL'].fmeasure
    }

def evaluar_modelos():
    resultados = cargar_resultados()
    modelos = ['mistral', 'llama3', 'phi', 'qwen25', 'deepseek']
    scores_por_modelo = {m: {'rouge1': [], 'rouge2': [], 'rougeL': []} for m in modelos}

    for entrada in resultados:
        texto_facilitado = entrada['facilitado']
        for modelo in modelos:
            if modelo in entrada and entrada[modelo] and not entrada[modelo].startswith('Error') and not entrada[modelo].isspace() and not entrada[modelo] == '':
                try:
                    rouge_scores = calcular_rouge(entrada[modelo], texto_facilitado)
                    for k in ['rouge1', 'rouge2', 'rougeL']:
                        scores_por_modelo[modelo][k].append(rouge_scores[k])
                except Exception as e:
                    print(f"Error calculando ROUGE para el modelo {modelo}: {e}")

    print("\nResultados de evaluación ROUGE (F1):")
    print("-" * 70)
    print("Modelo\t\tROUGE-1\t\tROUGE-2\t\tROUGE-L")
    print("-" * 70)
    for modelo in modelos:
        if scores_por_modelo[modelo]['rouge1']:
            r1 = np.mean(scores_por_modelo[modelo]['rouge1'])
            r2 = np.mean(scores_por_modelo[modelo]['rouge2'])
            rl = np.mean(scores_por_modelo[modelo]['rougeL'])
            print(f"{modelo:<12}\t{r1:.4f}\t\t{r2:.4f}\t\t{rl:.4f}")
        else:
            print(f"{modelo:<12}\tNo hay resultados válidos")

if __name__ == "__main__":
    evaluar_modelos()

## METEOR

In [ ]:
import json
import numpy as np
import nltk
from nltk.translate.meteor_score import meteor_score
try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/omw-1.4')
except LookupError:
    nltk.download('omw-1.4')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

def cargar_resultados(archivo='resultados.json'):
    with open(archivo, 'r', encoding='utf-8') as f:
        return json.load(f)

def calcular_meteor(texto_generado, texto_referencia):
    try:
        referencia_tok = nltk.word_tokenize(texto_referencia, language='spanish')
        generado_tok = nltk.word_tokenize(texto_generado, language='spanish')
    except:
        referencia_tok = nltk.word_tokenize(texto_referencia)
        generado_tok = nltk.word_tokenize(texto_generado)
    
    return meteor_score([referencia_tok], generado_tok)

def evaluar_modelos():
    resultados = cargar_resultados()
    modelos = ['mistral', 'llama3', 'phi', 'qwen25', 'deepseek']
    scores_por_modelo = {m: [] for m in modelos}
    
    for entrada in resultados:
        texto_facilitado = entrada['facilitado']
        
        for modelo in modelos:
            if (modelo in entrada and 
                entrada[modelo] and 
                not entrada[modelo].startswith('Error') and 
                not entrada[modelo].isspace() and 
                not entrada[modelo] == ''):
                
                try:
                    score = calcular_meteor(entrada[modelo], texto_facilitado)
                    scores_por_modelo[modelo].append(score)
                except Exception as e:
                    print(f"Error calculando METEOR para el modelo {modelo}: {e}")
    
    print("\nResultados de evaluación METEOR:")
    print("-" * 50)
    print("Modelo\t\tMedia\t\tDesv. Est.\tMín\t\tMáx")
    print("-" * 50)
    
    for modelo, scores in scores_por_modelo.items():
        if scores:
            stats = {
                'media': np.mean(scores),
                'std': np.std(scores),
                'min': np.min(scores),
                'max': np.max(scores)
            }
            print(f"{modelo:<12}\t{stats['media']:.4f}\t\t{stats['std']:.4f}\t\t{stats['min']:.4f}\t\t{stats['max']:.4f}")
        else:
            print(f"{modelo:<12}\tNo hay resultados válidos")

if __name__ == "__main__":
    evaluar_modelos()

## BERTScore

In [ ]:
import json
import numpy as np
from bert_score import score

BERT_MODEL = 'xlm-roberta-base'


def cargar_resultados(archivo='resultados.json'):
    with open(archivo, 'r', encoding='utf-8') as f:
        return json.load(f)

def calcular_bertscore_batch(candidatos, referencias):
    P, R, F1 = score(candidatos, referencias, lang='es', model_type=BERT_MODEL, verbose=False)
    return F1.tolist()

def evaluar_modelos():
    resultados = cargar_resultados()
    modelos = ['mistral', 'llama3', 'phi', 'qwen25', 'deepseek']
    scores_por_modelo = {m: [] for m in modelos}

    for modelo in modelos:
        candidatos = []
        referencias = []
        for entrada in resultados:
            texto_facilitado = entrada['facilitado']
            if modelo in entrada and entrada[modelo] and not entrada[modelo].startswith('Error') and not entrada[modelo].isspace() and not entrada[modelo] == '':
                candidatos.append(entrada[modelo])
                referencias.append(texto_facilitado)
        if candidatos:
            try:
                f1_scores = calcular_bertscore_batch(candidatos, referencias)
                scores_por_modelo[modelo].extend(f1_scores)
            except Exception as e:
                print(f"Error calculando BERTScore para el modelo {modelo}: {e}")

    print("\nResultados de evaluación BERTScore (F1):")
    print("-" * 50)
    print("Modelo\t\tMedia\t\tDesv. Est.\tMín\t\tMáx")
    print("-" * 50)
    for modelo, scores in scores_por_modelo.items():
        if scores:
            stats = {
                'media': np.mean(scores),
                'std': np.std(scores),
                'min': np.min(scores),
                'max': np.max(scores)
            }
            print(f"{modelo:<12}\t{stats['media']:.4f}\t\t{stats['std']:.4f}\t\t{stats['min']:.4f}\t\t{stats['max']:.4f}")
        else:
            print(f"{modelo:<12}\tNo hay resultados válidos")

if __name__ == "__main__":
    evaluar_modelos()